In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os 

# Imports added from modifying the original code
import random
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, Column, join
from hetdex_tools.get_spec import get_spectra
from sklearn.model_selection import train_test_split

Keras is a deep learning API written in Python, running on top of ther machine learning platform TensorFlow. Keras was developed with a focus on enabling fast experimentation.

So Keras is a high-level neural network library that runs on top of TensorFlow. Keras is more user friendly because it's built in Python.

In [36]:
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import History 

Comments and notes by me (Nick D) :) just to help me understand what's going on.

Callbacks: A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc)

Callbacks can be used to:
   * Monitor metrics by writing TensorBoard logs
   * Periodically save model to disk drive
   * Do early stopping
   * Get internal states and stats during training
   * and more!

In [37]:
# This is creating a 'History' callback object. This object keeps track of the accuracy, loss and other training metrics
# for each epoch in the memory.
# Not 100% sure, but it seems like this is what allows the code to make plots later on.
history = History()

In [38]:
# Not entirely sure what the number means. But I'm assuming it's purely a naming convention and does not
# mean anything specific.
training_index = "31_cut"

### Opening HDR3 Catalog and converting it into a Df

In [5]:
# Opening HDR3 detections catalog and converting it into a pandas DF
# HDR3 is detections HETDEX found
# Using HDR3 NEP and not H20 NEP because we can make SN cuts!
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# From the original DFs, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]

### Filtering data

In [6]:
# Removing data from before 2017 because it isn't good (not useful to us)
# No need to do this for H20 NEP
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
# Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.
# Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff
# We can start with high SN > 6, and slowly decrease to SN > 5
hdr3_signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_hdr3_df['sn'] > 6]


# Now I just take the sources stricly in the NEP field
hdr3_nep = hdr3_signal_to_noise_interval[hdr3_signal_to_noise_interval['field'] == 'nep']

Making skycoord object for hdr3 ra and dec

In [7]:
hdr3_skycoords = SkyCoord(hdr3_nep['ra'] * u.deg, hdr3_nep['dec'] * u.deg)

# Oscar's plan
### (Saving cause slack deletes messages sometimes)
Oh I do not think you want to input your noise sample through Valentina’s code, cause I do not think Valentina trained her encoder with that. What we would want to do is take a random sample of sources in the NEP field, run it through Valentina’s code and then select the output that her code says are high-z. Then we would run this “high-z” sample through your noise separator classification.
So it’ll be a two step process where you would test and train your ML algorithm using random forrest on LAEs and noise, optimize it so it has the highest accuracy possible.
Once you are comfortable with your ML algorithm then we do what i mentioned above, where we take some random sources, maybe we could apply some cuts to enhance the sample, in the NEP field run it through Valentina’s code select what her code says are high-z sources and then run those high-z sources through your ML algorithm and see which of those are actual LAEs and which are noise

# QUESTIONS:
1. Logging into TACC in public wifi? [x]

Yes!

2.  Is it ok to only use single spectras, or is that like not realistic data anymore? When I didn't filter for only 1 spectra, the shape of my array got weird and it didn't work with a lot of other things in the code.

# To DO:

1. Write code for checking for duplicates
    - Use wavelength to check, if its within a couple angstroms of each other.
    - Could use detectID as well.
    
2. Look into: "Why do we need a test and train set?"

### TAKES A LONG TIME. SKIP IF DONT HAVE TIME (have a saved version below) Extracts a random sample of sources in NEP. 

## QUESTION: Is it ok to only use single spectras? When we got more the shape of my array got weird and it didn't work with a lot of other things in the code.

Doesn't really matter, becuase in HETDEx there's only.

In [75]:
#random_nep_LS = []

# make it 1000 and let it run for awhile!!!
samples_amount = 1000

#random_nep_arr = np.zeros(samples_amount)

random_nep_LS = [[0] * 1 for i in range(samples_amount)]

#print(random_nep_LS)

counter = 0

while counter < samples_amount:
    #pick a random coordinate from the skycoord object
    random_coord = hdr3_skycoords[random.randint(0, hdr3_skycoords.size)]
    # extract spectra at this random coordinate and append to list, can't append to np array because of 
    # data type issues will convert to array after loop!
    extraction = get_spectra(random_coord)
    
    # HERE ALSO CHECK FOR DUPLICATES!!
    if len(extraction) == 1: 
    
        squeezed_extraction = np.squeeze(extraction['spec'].value)
        random_nep_LS[counter] = squeezed_extraction
        
        counter += 1
    else:
        continue
    
random_nep_spectra = np.array(random_nep_LS)

[INFO - 2022-11-18 07:49:59,926] Finding shots of interest
[INFO - 2022-11-18 07:50:06,181] Number of shots of interest: 6
[INFO - 2022-11-18 07:50:06,182] Extracting 6 sources
[INFO - 2022-11-18 07:50:06,469] Working on shot: 20210519023
[INFO - 2022-11-18 07:50:06,469] Working on shot: 20210518018
[INFO - 2022-11-18 07:50:06,551] Working on shot: 20210608012
[INFO - 2022-11-18 07:50:06,632] Working on shot: 20210609014
[INFO - 2022-11-18 07:50:06,724] Working on shot: 20210611012
[INFO - 2022-11-18 07:50:06,828] Working on shot: 20210612013
[INFO - 2022-11-18 07:50:10,328] Extracting 1
[INFO - 2022-11-18 07:50:11,193] Extraction of sources completed in 0.08 minutes.
[INFO - 2022-11-18 07:50:11,332] Retrieved 1 spectra.
[INFO - 2022-11-18 07:50:11,396] Finding shots of interest
[INFO - 2022-11-18 07:50:17,625] Number of shots of interest: 8
[INFO - 2022-11-18 07:50:17,626] Extracting 8 sources
[INFO - 2022-11-18 07:50:17,905] Working on shot: 20200421021
[INFO - 2022-11-18 07:50:17,91

[INFO - 2022-11-18 07:51:57,895] Working on shot: 20180831003
[INFO - 2022-11-18 07:51:57,979] Working on shot: 20180904009
[INFO - 2022-11-18 07:51:58,058] Working on shot: 20180910008
[INFO - 2022-11-18 07:51:58,138] Working on shot: 20180911011
[INFO - 2022-11-18 07:52:00,174] Extracting 1
[INFO - 2022-11-18 07:52:00,872] Extraction of sources completed in 0.06 minutes.
[INFO - 2022-11-18 07:52:01,008] Retrieved 1 spectra.
[INFO - 2022-11-18 07:52:01,070] Finding shots of interest
[INFO - 2022-11-18 07:52:07,443] Number of shots of interest: 3
[INFO - 2022-11-18 07:52:07,445] Extracting 3 sources
[INFO - 2022-11-18 07:52:07,641] Working on shot: 20190924008
[INFO - 2022-11-18 07:52:07,716] Working on shot: 20190926003
[INFO - 2022-11-18 07:52:07,800] Working on shot: 20190927010
[INFO - 2022-11-18 07:52:08,695] Extracting 1
[INFO - 2022-11-18 07:52:09,232] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-11-18 07:52:09,372] Retrieved 1 spectra.
[INFO - 2022-11-18 07:52:

[INFO - 2022-11-18 07:53:48,373] Working on shot: 20200522025
[INFO - 2022-11-18 07:53:48,451] Working on shot: 20200522026
[INFO - 2022-11-18 07:53:48,533] Working on shot: 20200711014
[INFO - 2022-11-18 07:53:48,612] Working on shot: 20200711015
[INFO - 2022-11-18 07:53:48,699] Working on shot: 20200712016
[INFO - 2022-11-18 07:53:48,777] Working on shot: 20200712017
[INFO - 2022-11-18 07:53:52,167] Extracting 1
[INFO - 2022-11-18 07:53:52,336] Extracting 1
[INFO - 2022-11-18 07:53:52,902] Extraction of sources completed in 0.08 minutes.
[INFO - 2022-11-18 07:53:53,041] Retrieved 2 spectra.
[INFO - 2022-11-18 07:53:53,106] Finding shots of interest
[INFO - 2022-11-18 07:53:59,256] Number of shots of interest: 11
[INFO - 2022-11-18 07:53:59,257] Extracting 11 sources
[INFO - 2022-11-18 07:53:59,482] Working on shot: 20180805010
[INFO - 2022-11-18 07:53:59,569] Working on shot: 20180806010
[INFO - 2022-11-18 07:53:59,670] Working on shot: 20180807008
[INFO - 2022-11-18 07:53:59,769] Wo

[INFO - 2022-11-18 07:55:15,888] Working on shot: 20210518021
[INFO - 2022-11-18 07:55:15,955] Working on shot: 20210519026
[INFO - 2022-11-18 07:55:16,041] Working on shot: 20210521022
[INFO - 2022-11-18 07:55:16,129] Working on shot: 20210605016
[INFO - 2022-11-18 07:55:16,263] Working on shot: 20210608015
[INFO - 2022-11-18 07:55:16,323] Working on shot: 20210608016
[INFO - 2022-11-18 07:55:16,456] Working on shot: 20210609016
[INFO - 2022-11-18 07:55:16,541] Working on shot: 20210610022
[INFO - 2022-11-18 07:55:16,589] Working on shot: 20210611015
[INFO - 2022-11-18 07:55:16,658] Working on shot: 20210612016
[INFO - 2022-11-18 07:55:16,737] Working on shot: 20210614018
[INFO - 2022-11-18 07:55:20,400] Extracting 1
[INFO - 2022-11-18 07:55:21,026] Extraction of sources completed in 0.09 minutes.
[INFO - 2022-11-18 07:55:21,209] Retrieved 1 spectra.
[INFO - 2022-11-18 07:55:21,271] Finding shots of interest
[INFO - 2022-11-18 07:55:27,452] Number of shots of interest: 2
[INFO - 2022-

[INFO - 2022-11-18 07:56:52,076] Extracting 1
[INFO - 2022-11-18 07:56:52,578] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-11-18 07:56:52,718] Retrieved 1 spectra.
[INFO - 2022-11-18 07:56:52,780] Finding shots of interest
[INFO - 2022-11-18 07:56:58,871] Number of shots of interest: 8
[INFO - 2022-11-18 07:56:58,872] Extracting 8 sources
[INFO - 2022-11-18 07:56:59,110] Working on shot: 20210518021
[INFO - 2022-11-18 07:56:59,209] Working on shot: 20210519025
[INFO - 2022-11-18 07:56:59,334] Working on shot: 20210519026
[INFO - 2022-11-18 07:56:59,400] Working on shot: 20210521022
[INFO - 2022-11-18 07:56:59,498] Working on shot: 20210605015
[INFO - 2022-11-18 07:56:59,658] Working on shot: 20210608014
[INFO - 2022-11-18 07:56:59,700] Working on shot: 20210610021
[INFO - 2022-11-18 07:56:59,812] Working on shot: 20210611014
[INFO - 2022-11-18 07:57:00,416] Extracting 1
[INFO - 2022-11-18 07:57:00,913] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-11-1

...

[INFO - 2022-11-18 10:50:55,534] Finding shots of interest
[INFO - 2022-11-18 10:51:01,393] Number of shots of interest: 4
[INFO - 2022-11-18 10:51:01,395] Extracting 4 sources
[INFO - 2022-11-18 10:51:01,665] Working on shot: 20210420019
[INFO - 2022-11-18 10:51:01,765] Working on shot: 20210420020
[INFO - 2022-11-18 10:51:01,864] Working on shot: 20210422018
[INFO - 2022-11-18 10:51:01,979] Working on shot: 20210506021
[INFO - 2022-11-18 10:51:02,990] Extracting 1
[INFO - 2022-11-18 10:51:03,514] Extraction of sources completed in 0.03 minutes.
[INFO - 2022-11-18 10:51:03,655] Retrieved 1 spectra.
[INFO - 2022-11-18 10:51:03,730] Finding shots of interest
[INFO - 2022-11-18 10:51:09,813] Number of shots of interest: 8
[INFO - 2022-11-18 10:51:09,814] Extracting 8 sources
[INFO - 2022-11-18 10:51:10,059] Working on shot: 20180805010
[INFO - 2022-11-18 10:51:10,176] Working on shot: 20180806010
[INFO - 2022-11-18 10:51:10,283] Working on shot: 20180807008
[INFO - 2022-11-18 10:51:10,39

[INFO - 2022-11-18 10:52:33,301] Number of shots of interest: 7
[INFO - 2022-11-18 10:52:33,302] Extracting 7 sources
[INFO - 2022-11-18 10:52:33,522] Working on shot: 20210611013
[INFO - 2022-11-18 10:52:33,594] Working on shot: 20210612014
[INFO - 2022-11-18 10:52:33,677] Working on shot: 20210616018
[INFO - 2022-11-18 10:52:33,740] Working on shot: 20210708012
[INFO - 2022-11-18 10:52:33,828] Working on shot: 20210709017
[INFO - 2022-11-18 10:52:33,902] Working on shot: 20210709018
[INFO - 2022-11-18 10:52:34,011] Working on shot: 20210713017
[INFO - 2022-11-18 10:52:37,798] Extracting 1
[INFO - 2022-11-18 10:52:38,345] Extraction of sources completed in 0.08 minutes.
[INFO - 2022-11-18 10:52:38,519] Retrieved 1 spectra.
[INFO - 2022-11-18 10:52:38,588] Finding shots of interest
[INFO - 2022-11-18 10:52:44,824] Number of shots of interest: 8
[INFO - 2022-11-18 10:52:44,826] Extracting 8 sources
[INFO - 2022-11-18 10:52:45,058] Working on shot: 20210605016
[INFO - 2022-11-18 10:52:45

[INFO - 2022-11-18 10:54:07,127] Working on shot: 20210609015
[INFO - 2022-11-18 10:54:07,215] Working on shot: 20210612015
[INFO - 2022-11-18 10:54:07,285] Working on shot: 20210616019
[INFO - 2022-11-18 10:54:07,359] Working on shot: 20210616020
[INFO - 2022-11-18 10:54:07,436] Working on shot: 20210616021
[INFO - 2022-11-18 10:54:07,518] Working on shot: 20210705015
[INFO - 2022-11-18 10:54:07,604] Working on shot: 20210709019
[INFO - 2022-11-18 10:54:10,150] Extracting 1
[INFO - 2022-11-18 10:54:10,663] Extraction of sources completed in 0.06 minutes.
[INFO - 2022-11-18 10:54:10,813] Retrieved 1 spectra.
[INFO - 2022-11-18 10:54:10,884] Finding shots of interest
[INFO - 2022-11-18 10:54:17,201] Number of shots of interest: 6
[INFO - 2022-11-18 10:54:17,203] Extracting 6 sources
[INFO - 2022-11-18 10:54:17,416] Working on shot: 20200713016
[INFO - 2022-11-18 10:54:17,483] Working on shot: 20200713017
[INFO - 2022-11-18 10:54:17,580] Working on shot: 20200723001
[INFO - 2022-11-18 10

[INFO - 2022-11-18 10:55:32,822] Retrieved 2 spectra.
[INFO - 2022-11-18 10:55:32,883] Finding shots of interest
[INFO - 2022-11-18 10:55:39,293] Number of shots of interest: 8
[INFO - 2022-11-18 10:55:39,294] Extracting 8 sources
[INFO - 2022-11-18 10:55:39,565] Working on shot: 20200813012
[INFO - 2022-11-18 10:55:39,660] Working on shot: 20200814012
[INFO - 2022-11-18 10:55:39,753] Working on shot: 20200814013
[INFO - 2022-11-18 10:55:39,868] Working on shot: 20200814014
[INFO - 2022-11-18 10:55:39,972] Working on shot: 20200815012
[INFO - 2022-11-18 10:55:40,070] Working on shot: 20200815013
[INFO - 2022-11-18 10:55:40,197] Working on shot: 20200815014
[INFO - 2022-11-18 10:55:40,287] Working on shot: 20200816011
[INFO - 2022-11-18 10:55:40,751] Extracting 1
[INFO - 2022-11-18 10:55:43,555] Extraction of sources completed in 0.07 minutes.
[INFO - 2022-11-18 10:55:43,703] Retrieved 1 spectra.
[INFO - 2022-11-18 10:55:43,786] Finding shots of interest
[INFO - 2022-11-18 10:55:49,786]

[INFO - 2022-11-18 10:57:02,559] Retrieved 1 spectra.
[INFO - 2022-11-18 10:57:02,627] Finding shots of interest
[INFO - 2022-11-18 10:57:08,822] Number of shots of interest: 8
[INFO - 2022-11-18 10:57:08,823] Extracting 8 sources
[INFO - 2022-11-18 10:57:09,040] Working on shot: 20200716013
[INFO - 2022-11-18 10:57:09,129] Working on shot: 20200816012
[INFO - 2022-11-18 10:57:09,196] Working on shot: 20200817011
[INFO - 2022-11-18 10:57:09,270] Working on shot: 20210711014
[INFO - 2022-11-18 10:57:09,349] Working on shot: 20210713016
[INFO - 2022-11-18 10:57:09,426] Working on shot: 20210728008
[INFO - 2022-11-18 10:57:09,513] Working on shot: 20210728009
[INFO - 2022-11-18 10:57:09,608] Working on shot: 20210827007
[INFO - 2022-11-18 10:57:10,039] Extracting 1
[INFO - 2022-11-18 10:57:12,627] Extraction of sources completed in 0.06 minutes.
[INFO - 2022-11-18 10:57:12,763] Retrieved 1 spectra.
[INFO - 2022-11-18 10:57:12,829] Finding shots of interest
[INFO - 2022-11-18 10:57:18,978]

In [83]:
random_nep_spectra.shape

(1000, 1036)

In [91]:
#np.save(f"random_nep_sample_{samples_amount}", random_nep_spectra)

In [92]:
# Loading in spectra sample
# The allow pickle part uses the pickle module which is not secure against maliciously constructed data
# Don't allow pickle if you don't trust the source, I do so it doesn't matter to me
# I have to do the allow_pickle = True for the code to not give me an error anyway
NEP_spec = np.load("random_nep_sample_1000.npy", allow_pickle = True)

In [93]:
NEP_spec.shape

(1000, 1036)

In [20]:
# Loading in spectra samples
star_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/star_spec_reduced.npy") 
agn_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/all_agn_no_duplicates.npy")
lowz_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/lowz_no_duplicates.npy")
gal_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/gal_sample.npy")

In [18]:
# This function checks for nan values.
# Interestingly enough, when you do nan != nan, you get True. Becuase NaN is unequal with anything.
# But with numbers, you'll get false.
def isNaN(num):
    return num != num

### Could also use numpy isnan

In [26]:
# If there are any values in the imports that are 'nan' (Not a Number) we remove them.
# Go through each value in matrix and check if NaN
def remove_nan(file):
    for i in range(0,len(file)):
        for j in range(0,len(file[i])):
            if isNaN(file[i][j]):

                # if value is nan, make the value equal to 0.0001
                # I'm assuming this was to avoid errors
                file[i][j]=0.00001

remove_nan(star_spec)
remove_nan(agn_spec)
remove_nan(lowz_spec)
remove_nan(gal_spec)
# Adding NEP files to the remove NaN function calls
remove_nan(NEP_spec)

In [32]:
# random.seed(2) will make the random numbers predictable, so random numbers, but the same random numbers from the same set/seed
# so I'll get the same random numbers everytime, until I change the seed
np.random.seed(2)
# random.shuffle will shuffle the values in an array or list. Like shuffling a deck of cards
# I wonder why she only shuffled starspec.
np.random.shuffle(star_spec)
np.random.shuffle(NEP_spec)

Tying to split between testing and training belo. scikit learning has something to make randomize data set and training sets.

In [52]:
# concatenate just joins arrays
all_images = np.concatenate((star_spec[::90],agn_spec,lowz_spec,gal_spec))
# making a training set
train_images = np.concatenate((star_spec[::90][:433],agn_spec[:444],lowz_spec[:837],gal_spec[:254]))
#train_images = agn_spec[:433]
#np.random.shuffle(train_images)

In [95]:
all_images.shape

(2458, 1036)

In [94]:
train_images.shape

(1968, 1036)

In [53]:
# not sure what these val_images are, it seems to be the opposite half of the train_images.
val_images = np.concatenate((star_spec[::90][433:],agn_spec[444:],lowz_spec[837:],gal_spec[254:]))
#val_images = agn_spec[433:]
#np.random.shuffle(val_images)

# Question: Is a 50/50 split between training and testing okay? Will look into how others do it, but wanted to ask here first. 

### Also, differece between training and testing? 

Largest trunk is training, validation is smaller, and test is smallest!

Training = 50

validation = 30

test = 20

Main difference only training on training set. Others are to test how well it works.

### My splitting between testing and training

In [57]:
# Here train_test_split takes in an array (NEP_spec)
# a test_size which is just a float from 0 to 1 to tell it how to split up the train and tests
# and random_state which works similar to the .shuffle. It helps control the shuffling and helps us reproduce outputs

NEP_train, NEP_test = train_test_split(NEP_spec, test_size=0.5, random_state=42)

In [58]:
NEP_train.shape

(50, 1036)

In [59]:
NEP_test.shape

(50, 1036)

In [60]:
# This function will normalize the data.
# The idea is, there is such a large range of sources so it may be simpler for the algorithm to recognize sources
# if they are between 0 and 1.
# Dividing by the largest number preserves the information but scales it down.
# If we don't do this the sources could go from a very large number to a small one, which makes it harder for the algorithm
def Norm(data):
    new_data = []
    for spec in data:
        #print('spec_before', spec)
        spec = spec/spec.max()
        #print('spec_after', spec)
        #print('spec.max', spec.max())
        new_data.append(spec)
    return np.array(new_data)

In [61]:
# Normalizing both data sets
train_images = Norm(train_images)
val_images = Norm(val_images)

NEP_train = Norm(NEP_train)
NEP_test = Norm(NEP_test)

In [65]:
train_images.shape

(1968, 1036)

# QUESTION: What do I do here? Do I need to cut anything?

In [62]:
# Making cuts of the image sets and then normalizing them.
train_cut = train_images[:,88:1002]
val_cut = val_images[:,88:1002]
train_cut = Norm(train_cut)
val_cut = Norm(val_cut)

Cutting out left and right ends, because when making inspection there can be weird things on the left most end.

In [66]:
train_cut.shape

(1968, 914)

In [67]:
latent_dim = 30
act = 'sigmoid'

In [69]:
class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
          #layers.InputLayer(input_shape=(1036)),
          layers.Dense(436, activation=act,name="e1"),
          #layers.Dropout(0.3),
          #layers.Dense(136, activation=act, name ='e2'),
          #BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(latent_dim, activation=None, name = "e3"),
        ])
        self.decoder = tf.keras.Sequential([
          #layers.InputLayer(input_shape=(latent_dim,)),
          #layers.Dense(136, activation=act,name = "d1"),
          layers.Dense(436, activation=act,name = "d2"),
          #BatchNormalization(),
          layers.Dense(914, activation=None,name = "d3")
        ])#914 for cut. 1036 for all

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [70]:
autoencoder = Autoencoder(latent_dim)
#autoencoder.build(input_shape=np.shape(train_images))
epoch = 500

2022-11-18 07:23:38.944014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-18 07:23:38.944264: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-18 07:23:38.944415: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-nimda16): /proc/driver/nvidia/version does not exist
2022-11-18 07:23:38.949140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [71]:
adg = tf.keras.optimizers.Adagrad(
    learning_rate=0.01)

In [72]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0005)

In [ ]:
def loss1(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    d = tf.reduce_mean(tf.math.square(y_pred - y_true), axis=-1)
    #tf.print(d, [d], "Inside loss function")
    #tf.print(d.shape)
    return d

autoencoder.compile(optimizer=adam, loss=loss1)#es.MeanSquaredError())

try:
    os.mkdir(f"models/training{training_index}")
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
except:
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

# QUESTION: What do I do for validation data and for training data? Confused on this part. 

Read up a bit: "In autoencoder, this validation_data set is used just for calculating the loss function. So, the test error of an autocencoder is nothing but the output of your selected loss function on this validation set." 

"Validation data is not used for training (or development of the model). Its purpose is to track progress through validation loss and accuracy."

Link to read further: https://github.com/keras-team/keras/issues/1753

In [ ]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#autoencoder.summary()
#autoencoder.fit(train_images, train_images,
 #               epochs=10,
  #              shuffle=True,
   #             validation_data=(val_images,val_images))
hist = autoencoder.fit(train_cut, train_cut,
                epochs=epoch,
                shuffle=True,
                validation_data=(val_cut,val_cut),
                callbacks=[cp_callback])


In [ ]:
plt.figure()
x = np.linspace(1,epoch+1,epoch)
plt.plot(x,hist.history["loss"],label="loss")
plt.plot(x,hist.history["val_loss"],label="val loss")
plt.legend()
plt.savefig(f'Graphs/loss_{training_index}.png')
print(np.shape(train_images))

encoded_imgs = autoencoder.encoder(train_cut).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
np.save(f"encoded_{training_index}",encoded_imgs)
np.save(f"decoded_{training_index}",decoded_imgs)

encoded_val = autoencoder.encoder(val_cut).numpy()
np.save(f"encoded_val_{training_index}",encoded_val)

In [ ]:
np.load("encoded_31_cut.npy")

In [ ]:
plt.figure()
plt.plot(train_cut[4])
plt.savefig("OG.png")
plt.figure()
plt.plot(decoded_imgs[4])
plt.savefig("RECON.png")
print(hist.history["val_loss"][-1])